In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d sartajbhuvaji/brain-tumor-classification-mri

In [ ]:
!unzip brain-tumor-classification-mri.zip

In [ ]:
!pip install utils

In [ ]:
import os
import cv2
import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [ ]:
import ipywidgets as widgets
import io
from PIL import Image
import tqdm
import tensorflow as tf

## Folder Paths

In [ ]:
X_train = []
Y_train = []
image_size = 256
labels = ["glioma_tumor","meningioma_tumor","no_tumor","pituitary_tumor"]
for i in labels:
  folderPath = os.path.join("/content/Training",i)
  for j in os.listdir(folderPath):
    img = cv2.imread(os.path.join(folderPath, j))
    img = cv2.resize(img,(image_size,image_size))
    X_train.append(img)
    Y_train.append(i)


for i in labels:
  folderPath = os.path.join("/content/Testing",i)
  for j in os.listdir(folderPath):
    img = cv2.imread(os.path.join(folderPath, j))
    img = cv2.resize(img,(image_size,image_size))
    X_train.append(img)
    Y_train.append(i)


X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
X_train, Y_train = shuffle(X_train, Y_train, random_state= 101)
X_train.shape

## Train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train,Y_train, test_size=0.1, random_state=101)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
y_train_new = []
for i in y_train:
  y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
  y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

## Convolutional Neural Network

In [ ]:
model = Sequential()

model.add(Conv2D(32,(3,3),activation = "relu", input_shape=(256,256,3)))
model.add(Conv2D(64,(3,3),activation = "relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(64,(3,3),activation = "relu"))
model.add(Conv2D(64,(3,3),activation = "relu"))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(128,(3,3),activation = "relu"))
model.add(Conv2D(128,(3,3),activation = "relu"))
model.add(Conv2D(256,(3,3),activation = "relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Conv2D(256,(3,3),activation = "relu"))
model.add(Conv2D(256,(3,3),activation = "relu"))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(512,activation = "relu"))
model.add(Dense(512,activation = "relu"))
model.add(Dropout(0.3))

model.add(Dense(4, activation = "softmax"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs = 50, validation_split=0.1)

In [ ]:
model.save("braintumor.h5")

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
epochs = range(len(acc))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,acc, "r", label="Training Accuracy")
plt.plot(epochs,val_acc, "b", label="Validation Accuracy")
plt.legend(loc="upper left")
plt.show()

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(len(loss))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,loss, "r", label="Training Loss")
plt.plot(epochs,val_loss, "b", label="Validation Loss")
plt.legend(loc="upper left")
plt.show()

## Prediction

In [ ]:
img = cv2.imread("/content/Training/pituitary_tumor/p (464).jpg")
img = cv2.resize(img,(256,256))
img_array = np.array(img)
img_array.shape

In [ ]:
img_array = img_array.reshape(1,256,256,3)
img_array.shape

In [ ]:
from tensorflow.keras.preprocessing import image
img = image.load_img("/content/Training/pituitary_tumor/p (464).jpg")
plt.imshow(img, interpolation="nearest")
plt.show()

In [ ]:
a = model.predict(img_array)
indices = a.argmax()
indices